# AI - Computer Assignment 2

## Genetic

In this assignment we are going to find the key for a particular Substitution Cipher problem using genetic algorithm. As we know there are genes, chromosomes and population in genetic algorithm.


### Genes and Chromosomes
Chromosomes are made of genes. Each individual chromosome is a member of our seach space. As we are going to find the key for Substitution Cipher our chromosomes are keys and are made of english alphabets. So our genes are characters and each chromosome is representing a mapping of lowercase alphabets to the corresponding letter in the cipher text.

The following block is definition of the Chromosome class.

In [2]:
from collections import Counter
from math import log2
from random import choice, random, sample
from re import findall, split
from string import ascii_lowercase, ascii_uppercase
from time import time
from typing import List

DEFAULT_POPULATION_SIZE = 100
MUTATION_PROBABILITY = 0.4
CROSSOVER_PROBABILITY = 0.5
MAX_REPEAT_COUNT = 100


def shuffle_str(s: str) -> str:
    return ''.join(sample(list(s), len(s)))


class Chromosome(object):

    def __init__(self, mapping: str):
        self.mapping = {
            k: v for k, v in zip(list(ascii_lowercase), list(mapping))
        }

    def decode_char(self, encoded_char):
        if encoded_char.isalpha():
            if encoded_char.isupper():
                return self.mapping.get(encoded_char.lower()).upper()
            else:
                return self.mapping.get(encoded_char)
        else:
            return encoded_char

    def decode(self, encoded_text):
        return ''.join([self.decode_char(char) for char in encoded_text])

    @classmethod
    def random(cls):
        return cls(shuffle_str(ascii_lowercase))

    @classmethod
    def crossover(cls, father, mother):
        def generate_mapping(father, mother, first_point, second_point, max_len):
            mapping = [' '] * max_len
            for i in range(first_point, second_point):
                mapping[i] = father.mapping.get(ascii_lowercase[i])

            i = j = second_point
            while j != first_point:
                mother_mapping = mother.mapping.get(ascii_lowercase[i])
                if mother_mapping not in mapping:
                    mapping[j] = mother_mapping
                    j = (j + 1) % max_len

                i = (i + 1) % max_len

            if random() < MUTATION_PROBABILITY:
                x, y = sorted(
                    sample(range(max_len), 2)
                )
                mapping[x], mapping[y] = mapping[y], mapping[x]

            return ''.join(mapping)

        if random() < CROSSOVER_PROBABILITY:
            max_len = len(ascii_lowercase)
            first_point, second_point = sorted(
                sample(range(max_len), 2)
            )

            boy_mapping = generate_mapping(
                father, mother, first_point, second_point, max_len
            )
            girl_mapping = generate_mapping(
                mother, father, first_point, second_point, max_len
            )

            return cls(boy_mapping), cls(girl_mapping)
        else:
            return father, mother

    def __str__(self):
        return ''.join(self.mapping.values())



The only parameter to the constructor is a string of english alphabets as the key. The decode methods simply decodes the given text as parameter.

There 2 classmethods which are custom constructors. The random clsaamethods gives a Chromosome with a random key. The crossover classmethod uses the [OX1 algorithm](https://www.tutorialspoint.com/genetic_algorithms/genetic_algorithms_crossover.htm) to genereate offsprings. With a probability of `CROSSOVER_PROBABILITY` it generates 2 new Chromosem based on the algorithm and returns the parents otherwise. In case of generating offsprings it also as a chance of `MUTATION_PROBABILITY` to mutate the generated offsprings by swapping 2 random characters of the mapping.

### Decoder
The Decoder class has the responsibility to decode the given text.

In [ ]:
def ngram(text: str, n: int = 2):

    def pairwise(s: str, n):
        n -= 1
        prev = s[:n]

        for item in s[n:]:
            yield prev + item
            prev = prev[1:] + item

    counter = Counter()
    words = split('\W+', text.lower())

    for word in words:
        for pair in pairwise(word, n):
            counter[pair] += 1

    return counter


class Decoder(object):

    def __init__(self, encoded_text, population_size: int = DEFAULT_POPULATION_SIZE):
        self.encoded_text: str = encoded_text
        self.ref_ngram = ngram(text=open('global_text.txt').read())
        self.population: List[Chromosome] = [
            Chromosome.random() for i in range(population_size)
        ]
        self.population_size: int = population_size
        self.fitness_cache = {}

    def calculate_fitness(self, chromosome: Chromosome):
        mapping = str(chromosome)
        if mapping in self.fitness_cache:
            return self.fitness_cache.get(mapping)

        fitness = 0.0
        decoded_ngram = ngram(
            text=chromosome.decode(self.encoded_text).lower()
        )
        for pair, occurrences in decoded_ngram.items():
            fitness += occurrences * log2(self.ref_ngram[pair] or 1)

        self.fitness_cache[mapping] = fitness
        return fitness

    def decode(self) -> str:
        found = False
        generation = 0
        start = time()

        repeat_count = 0
        best_fitness = 0

        while True:
            population = sorted(
                self.population,
                key=self.calculate_fitness,
                reverse=True
            )

            best_chromosome = population[0]
            current_best_fitness = self.calculate_fitness(best_chromosome)
            if current_best_fitness > best_fitness:
                best_fitness = current_best_fitness
                repeat_count = 0
            elif current_best_fitness == best_fitness:
                repeat_count += 1

            print(generation, current_best_fitness, best_chromosome)
            if repeat_count > MAX_REPEAT_COUNT:
                print("Key:", best_chromosome)
                print("Elapsed Time:", time() - start)
                return best_chromosome.decode(self.encoded_text)

            self.population = []
            for i, chromosome in enumerate(population):
                if len(self.population) >= self.population_size:
                    break

                if i < (self.population_size / 5):
                    self.population.append(chromosome)
                else:
                    father, mother = sample(population[:20], 2)
                    boy, girl = Chromosome.crossover(father, mother)
                    self.population.append(boy)
                    self.population.append(girl)

            generation += 1


### Population
It gets the encoded text and initial population size as constructor arguments. It also has a default value of `DEFAULT_POPULATION_SIZE` for the initial population size. Then it generates random chromosomes as the initial population.

### Fitness
The decoder uses ngrams to calculate the fitness of chromosomes. As the refrence it counts the occurrences of bigrams in the given global text. The words with one character are omitted from the text automatically as they have no bigrams. We can also omit the stop words from the text but i have not tried that.

The fitness of chromosome x is calculated using following formula:
$$\sum_{y \in N(x)} log_2(R[y]) * N(x)[y]$$
where N(x) is the frequency of bigrams found in the decrypted text using chromosome x and R is the frequency of bigrams in the global text. The logarithm function is used to deal with sparse distributions of bigram frequencies in the global text.

The higher the fitness is, the the better the chromosome is.

### Decode
To decode the text we start by the initial population of random chromosomes and at each iteration we sort them by the calculated fitness. We also count number of iterations with unchanged best fitness, if the count gets bigger than `MAX_REPEAT_COUNT`, the serach ends. Otherwise the top 20% of the population based on the fitness goes to the next generation and the rest of generation is populated using the crossover function explained above. At last the decoded text using the chromosome with highest fitness is returned.

### Mutation vs Crossover
The mutation helps us prevent the convergence of chromosomes to a local maximum fitness. The mutation swaps 2 characters of the generated offsprings to create new chromosomes to give the population a chance to find better chromosomes with higher fitness.